In [1]:
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import json
import pickle
import csv
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import networkx as nx
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string
import re
import pandas
import pandas as pd
import tweepy
import base64
import datetime
import preprocessor as p
from tweepy import Stream
from sklearn.externals import joblib
from nltk import ngrams
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from node2vec import Node2Vec

Using TensorFlow backend.
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sakshi18013/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sakshi18013/anaconda3/lib/python3.7

In [2]:
train_data=dict()

In [3]:
def getRandomtweetAPI():
    import random
    df = pandas.read_csv(os.getcwd()+'/tokens_random.csv',encoding='ISO-8859-1',keep_default_na=False)    
    totalTokens=df.shape[0]
    index=random.randint(0,totalTokens-1)   
    ckey=df['ConsumerKey'][index]
    csecret=df['ConsumerSecret'][index]
    atoken=df['TwitterToken'][index]
    asecret=df['TwitterSecret'][index]
    app_only_auth: true
    return ckey,csecret,atoken,asecret

In [4]:
def get_api():
    ckey,csecret,atoken,asecret=getRandomtweetAPI()
    auth = OAuthHandler(ckey, csecret)
    auth.set_access_token(atoken, asecret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

# Prepare Dataset

In [2]:

with open('follow_network_with_id.pkl', 'rb') as handle:
    foll_id = pickle.load(handle)

In [6]:
# rts[1230411884202868737]

In [3]:

with open('Retweeter_dict.pickle', 'rb') as handle:
    rts = pickle.load(handle)

In [8]:
# with open('tweet_features.pickle', 'rb') as handle:
#     tweet_features = pickle.load(handle)
# retweet_features=dict()
# retweeters_dataset_dict=dict()

In [9]:
G= nx.DiGraph()
import copy

In [12]:
# FOLDERNAME1="twitter_Retweeters_history"
# FOLDERNAME2="twitter_Followers_history"
# rts_dataset=dict()
# total=0
# labels=[]
# ct=0
# for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
#     for fname in files:
# #         print(fname)
#         if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
#             filepath=root+"/"+fname
#             search_file=filepath.split("/")
# #             print("here")
#             if len(search_file)>7:
# #                 print(search_file[7])
#                 tweets=dict()
#                 with open(filepath) as csv_file:
#                     csv_reader= csv.DictReader(csv_file)
#                     for row in csv_reader:

#                         inner={}
#                         if  int(row['retweet_count'])>5:
# #                             row['id_'] in tweet_features and
                            
#                             if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
#                                 retweeters = rts[int(row['id_'])]
                
#                                 tobecovered=copy.deepcopy(retweeters)
# # first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']))                       

# ####FOR FIRS LEVEL FOLLOWERS
# # for value in rts[int(row['id_'])]:
#                                 post_user = row['user_id']
#                                 for value in foll_id[int(row['user_id'])]:
# #                                     print(post_user, value)
                    
#                                     G.add_edge(post_user , value)
                                  
# ####FOR SECOND LEVEL FOLLOWERS  
# #                                 print(type(foll_id[int(row['user_id'])]), type(retweeters))
#                                 level1 = list(set(foll_id[int(row['user_id'])]) and  set(retweeters))
# #                                 print((level1) , "First level")
#                                 l2=list(set(retweeters)-set(level1))
# #                                 print(l2)
# #                                 second_retweets=[]
# #                                 print(len(level)
                                
#                                 l2_covered=[]
#                                 if l2:
#                                     for user in level1:
#     # #                                     print(" user",user)
#                                         if int(user) in foll_id:

#                                             for value1 in foll_id[int(user)]:
#     #                                                 print("here")
#                                                     if value1 in l2:
#                                                         l2_covered.append(value1)
#                                                     G.add_edge(user ,value1)
#                                                     ct+=1
                
                                
#         # ####THIRD LEVEL FOLLOWERS
#                                     level3 = list(set(l2)-set(l2_covered))
#     #                                 third_retweets=[]
#     # #                                 print("IN LEVEL 3",level3)
#                                     print(len(level3) , "Secomd level")
#                                     l3 =list( set(retweeters )- set(level3))

#                                     l3_covered=[]
#                                     if l3:
#                                         for user3 in level3:
#                                             if int(user3) in foll_id:

#                                                 for value2 in foll_id[int(user3)]:
#         #                                                 print("here")
#                                                         if value2 in l3:
#                                                             l3_covered.append(value2)
#                                                         G.add_edge(user3 ,value2)
#                                                         ct+=1

                                
#                                         level4= list(set(l3)-set(l3_covered))
#                                         G.add_node("Dummy")
#                                         G.add_edge(post_user , "Dummy")
#                                         for v in level4:
#                                             G.add_edge("Dummy", v)
#                                             ct+=1
#                                 print("Total number of retweeters:" , len(retweeters) ,"\n remainng nodes dumy connected" , len(retweeters)-ct)
#         #                                     
    
# #                     break 

In [4]:
G= nx.DiGraph()
import copy

In [ ]:
# G= nx.DiGraph()
# FOLDERNAME1="twitter_Retweeters_history"
# FOLDERNAME2="twitter_Followers_history"
# rts_dataset=dict()
# total=0
# # labels=[]

# tweets_ids = []
# for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
#     for fname in files:
# #         print(fname)
#         if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
#             filepath=root+"/"+fname
#             search_file=filepath.split("/")
# #             print("here")
#             if len(search_file)>7:
# #                 print(search_file[7])
#                 tweets=dict()
#                 with open(filepath) as csv_file:
#                     csv_reader = csv.DictReader(csv_file)
#                     for row in csv_reader:
#                         tweets_ids.append(row['id_'])
#                         inner={}
#                         if  int(row['retweet_count'])>5:
# #                             row['id_'] in tweet_features and
#                             G1= nx.DiGraph()
#                             if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
#                                 retweeters = rts[int(row['id_'])]
#                                 tobecovered=[]
#                                 tobecovered=copy.deepcopy(retweeters)
# # first_retweets=[]
# #                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       

# ####FOR FIRS LEVEL FOLLOWERS
# # for value in rts[int(row['id_'])]:
#                                 post_user = row['user_id']
#                                 for value in foll_id[int(row['user_id'])]:
# #                                     print(post_user, value)
#                                     try:
        
#                                         G1.add_edge(post_user , value)
#                                     except:
#                                         pass
                                  
# ####FOR SECOND LEVEL FOLLOWERS  
# #                                 print(type(foll_id[int(row['user_id'])]), type(retweeters))
#                                 level1 = list(set(foll_id[int(row['user_id'])]) and  set(retweeters))
# #                                 print("covered in level1",len(level1))
# #                                 print((level1) , "First level")
#                                 l2=list(set(retweeters)-set(level1))
#                                 for i in level1: 
#                                     try: 
#                                         tobecovered.remove(i) 
#                                     except ValueError: 
#                                         pass
# #                                 tobecovered.remove(level1)
# #                                 if(len(tobecovered)>0):
# #                                     print(len(tobecovered),"rem 2")
# #                                 print(len(l2), len(tobecovered) , "check")
# #                                 second_retweets=[]
# #                                 print(len(level)
                                
#                                 l2_covered=[]
#                                 if tobecovered:
# #                                     print("in")
#                                     for user in level1:
#     # #                                     print(" user",user)
#                                         if int(user) in foll_id:
# #                                             print("got")

#                                             for value1 in foll_id[int(user)]:
#     #                                                 print("here")
#                                                     if value1 in tobecovered:
#                                                         l2_covered.append(value1)
# #                                                     print("adding")
#                                                     G1.add_edge(user ,value1)
# #                                                     if nx.is_
                
#                                     for i in l2_covered: 
#                                         try: 
#                                             tobecovered.remove(i) 
#                                         except ValueError: 
#                                             pass
# #                                     if(len(tobecovered)>0):
# #                                         print(len(tobecovered),"rem 3")
# #                                     tobecovered.remove(l2_covered)
#         # ####THIRD LEVEL FOLLOWERS
#                                     level3 = list(set(l2)-set(l2_covered))
#     #                                 third_retweets=[]
#     # #                                 print("IN LEVEL 3",level3)
# #                                     print(len(level3) , "Secomd level")
#                                     l3 =list( set(retweeters )- set(level3))
#                                     for i in level3: 
#                                         try: 
#                                             tobecovered.remove(i) 
#                                         except ValueError: 
#                                             pass
# #                                     tobecovered.remove(level3)
#                                     l3_covered=[]
#                                     if tobecovered:
#                                         for user3 in level3:
#                                             if int(user3) in foll_id:

#                                                 for value2 in foll_id[int(user3)]:
#         #                                                 print("here")
#                                                         if value2 in tobecovered:
#                                                             l3_covered.append(value2)
#                                                         G1.add_edge(user3 ,value2)
#                                         for i in l3_covered: 
#                                             try: 
# #                                                 print("try")
#                                                 tobecovered.remove(i) 
#                                             except ValueError: 
#                                                 pass
                                    
# #                                         tobecovered.remove(l3_covered)
# #                                         if(len(tobecovered)>0):
# #                                             print(len(tobecovered),"rem 4")
                                
#                                         level4= list(set(l3)-set(l3_covered))
#                                         if level4:
#                                             G1.add_node("Dummy")
#                                             G1.add_edge(post_user , "Dummy")
#                                             for v in level4:
#                                                 G1.add_edge("Dummy", v)
#                                             for i in level4: 
#                                                 try: 
#                                                     tobecovered.remove(i) 
#                                                 except ValueError: 
#                                                     pass
# #                                             tobecovered.remove(level4)
#                                             print("Dummy Nodes:" , len(level4))
#                                 print("Total number of retweeters:" , len(retweeters) )
# #         ,"\n remainng nodes dumy connected" , len(level4))
# #                                 print("Remaining: " , len(tobecovered))              
    
# #                     break 
#                             print(len(G1.nodes), len(G1.edges()) , nx.is_directed_acyclic_graph(G1))
#                             for v , d in G1.in_degree() :
#                                 if d==0:
#                                     print(v)
#                             roots = (v for v, d in G1.in_degree() if d == 0)
#                             leaves = (v for v, d in G1.out_degree() if d == 0)

#                             for root in roots:
#                                 print("Root:",root)
#                                 for leaf in leaves:
# #                                     print("Leaf",leaf)
#                                     paths = nx.all_simple_paths(G1, root, leaf)
#                                     all_paths.extend(paths)
#     #                                 break
#                             print(len(all_paths))
# #                             break
                        

In [15]:
print(len(G.nodes()) ,len(G.edges()))

0 0


In [16]:
# print(all_paths[26845:29582])

In [79]:
with open('Retweeter_dict.pickle', 'rb') as handle:
    rts= pickle.load(handle)

In [78]:
# import pick.le
ids = pickle.load(open("tweet_ids_jamia","rb"))

In [82]:
ground_truth=[]
for key,value in rts.items():
    if str(key) in ids:
        ground_truth.extend(value)

In [89]:
g_truth_set =list(set(ground_truth))

In [90]:
pickle.dump(g_truth_set,open("ground_truth_jamia","wb"))

# GRAPH CREATION

In [53]:
G= nx.DiGraph()
FOLDERNAME1="twitter_Retweeters_history"
FOLDERNAME2="twitter_Followers_history"
rts_dataset=dict()
total=0
labels=[]
all_paths = []
all_path2=[]
final_cascade =[]
final_test_cascade=[]
max_=0
count_=0
user=[]
for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
    for fname in files:
#         print(fname)
        if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
            filepath=root+"/"+fname
            search_file=filepath.split("/")
#             print("here")
            if len(search_file)>7:
#                 print(search_file[7])
                tweets=dict()
                with open(filepath) as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
                        
                        inner={}
                        if  int(row['retweet_count'])>35:
#                             row['id_'] in tweet_features and
#                             G1= nx.DiGraph()
#                             G2 = nx.DiGraph()

                            if int(row['user_id']) in foll_id  and int(row['id_']) in rts:
                                retweeters = rts[int(row['id_'])]
#                                 tobecovered=set()
                                tobecovered=set(copy.deepcopy(retweeters))
# first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
####FOR FIRS LEVEL FOLLOWERS
# for value in rts[int(row['id_'])]:
                                post_user =row['user_id']
                                user.append(post_user)
                                print(post_user)
                                for value in foll_id[int(row['user_id'])]:
#                                     print(post_user, value)
                                    try:
        
                                        G.add_edge(post_user , value)
                                    except:
                                        pass
                    
                                upper_level = set([post_user])
                                flag = False
            
                                iter_=1
#                                 print("before loop: {}".format(foll_id[int(post_user)]))
                                
                                while(not flag and tobecovered):
                                    print("iter: {}".format(iter_))
                                    iter_+=1
                                    flag=True
                                    nowcovered=set()
#                                     followers=set()
                                    for each_user in upper_level: 
                                        if int(each_user) in foll_id:
                                            for each_follower in foll_id[int(each_user)]:
#                                                 followers.add(each_follower)
#                                                 G.add_edge(each_user , each_follower)
                                                if each_follower in tobecovered:
                                                    nowcovered.add(each_follower)
                                                    flag=False
                                                    G.add_edge(each_user , each_follower)
                                    for each_covered in nowcovered:
                                        tobecovered.remove(each_covered)
                                    upper_level = nowcovered
#                                 print(flag , len(tobecovered),"after while")
                                
                                if tobecovered:
                                    for v in tobecovered:
                                        G.add_node(v)
                                    upper_level = copy.deepcopy(tobecovered)
                                    flag = False
                                    iterg_=1
    #                                 print("before loop: {}".format(foll_id[int(post_user)]))

                                    while(not flag and tobecovered):
                                        print("iter: {}".format(iterg_))
                                        iterg_+=1
                                        flag=True
                                        nowcovered=set()
#                                         followers=set()
                                        for each_user in upper_level:
                                            if int(each_user) in foll_id:
                                                for each_follower in foll_id[int(each_user)]:
#                                                     followers.add(each_follower)
#                                                     G.add_edge(each_user , each_follower)
                                                    if each_follower in tobecovered:
                                                        nowcovered.add(each_follower)
                                                        flag=False
                                                        G.add_edge(each_user , each_follower)
                                        for each_covered in nowcovered:
                                            tobecovered.remove(each_covered)
                                        upper_level = nowcovered
#                                     print(flag , len(tobecovered),"after 2nd while")
                                    
                                    


#                             roots = (v for v, d in G1.in_degree() if d == 0)
#                             leaves = (v for v, d in G1.out_degree() if d == 0)

#                             for root in roots:
# #                                 print("Root:",root)
#                                 for leaf in leaves:
# #                                     print("Leaf",leaf)
#                                     paths = nx.all_simple_paths(G1, root, leaf)
#                                     all_paths.extend(paths)
#     #                                 break
#                             print("Actual cacades:" , len(all_paths))
        
        
#                             roots = (v for v, d in G2.in_degree() if d == 0)
#                             leaves = (v for v, d in G2.out_degree() if d == 0)

#                             for root in roots:
# #                                 print("Root:",root)
#                                 for leaf in leaves:
# #                                     print("Leaf",leaf)
#                                     paths = nx.all_simple_paths(G2, root, leaf)
#                                     all_path2.extend(paths)
#     #                                 break
#                             max_+=len(all_path2)
#                             count_+=1
#                             print(max_)
#     #                             break
    #                             dummy_path = all_path2 + list(tobecovered)
    #                             print("Dummy cascade" , len(dummy_path))
#                             print(dummy_path)


#                             for actual in all_paths:
#                                 final_test_cascade.append(actual)
#                                 if len(all_path2)==0  or len(all_path2)<71:
#                                     final_cascade.append(actual)
#                                 else:
#                                     for dummy in all_path2:
#                                         temp=list()
#                                         temp.extend(actual)
#                                         temp.extend(dummy)
#                                         final_cascade.append(temp)

#                             break
                        

279483060
iter: 1
iter: 2
iter: 1
1200095151751778309
iter: 1
iter: 2
iter: 3
iter: 1
1021495661013942278
iter: 1
iter: 2
iter: 1
4358041572
iter: 1
iter: 2
iter: 1
743275561
iter: 1
iter: 2
iter: 1
iter: 2
1213812904413122563
iter: 1
iter: 2
iter: 3
iter: 1
859309618270322688
iter: 1
iter: 2
iter: 1
279483060
iter: 1
iter: 2
iter: 1
773727446285819905
iter: 1
iter: 2
iter: 1
560621752
iter: 1
iter: 1
iter: 2
2695674967
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
2540896328
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
908667324911243264
iter: 1
iter: 1
iter: 2
4444559413
iter: 1
iter: 2
iter: 1
iter: 2
938658666089037830
iter: 1
iter: 2
iter: 3
iter: 1
938658666089037830
iter: 1
iter: 2
iter: 1
2826529099
iter: 1
iter: 2
iter: 3
iter: 1
iter: 2
1213812904413122563
iter: 1
iter: 2
iter: 3
iter: 1
938658666089037830
iter: 1
iter: 2
iter: 3
iter: 1
1185548380845404163
iter: 1
iter: 2
iter: 1
279483060
iter: 1
iter: 2
iter: 1
1225476466860650497
iter: 1
iter: 1
141566388
iter: 1
iter: 2
iter: 1
ite

In [57]:
print(len(user) , len(G.nodes()), len(G.edges()))

29 239951 291391


In [58]:
G= nx.DiGraph()
import copy
FOLDERNAME1="twitter_Retweeters_history"
FOLDERNAME2="twitter_Followers_history"
rts_dataset=dict()
total=0
labels=[]
# final_data=[]

# final_cascade =[]
# final_test_cascade=[]
users=[]
m1=0
for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
    for fname in files:
#         print(fname)
        if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
            filepath=root+"/"+fname
            search_file=filepath.split("/")
#             print("here")
            if len(search_file)>7:
#                 print(search_file[7])
#                 tweets=dict()
                with open(filepath) as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
#                         print("====Row===={}".format(row))
                        inner={}
                        if  int(row['retweet_count'])>5:
#                             row['id_'] in tweet_features and
#                             G1= nx.DiGraph()
#                             G2 = nx.DiGraph()
#                             all_paths = []
#                             all_path2=[]
                            retweeters=[]
                            if int(row['user_id']) in foll_id  and int(row['id_']) in rts_time:
                                retweeters=[]
                                retweeters = rts[int(row['id_'])]
#                                 print("i",row['user_id'],'3860520309' in retweeters)
#                                 retweeters.append()
                                tobecovered=set()
#                                 print(type(retweeters))
                                tobecovered=set(copy.deepcopy(retweeters))
#                                 print(len(retweeters), "tobecovered vefore")
#                                 print(retweeters)
                                nowcovered=set()
# first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
####FOR FIRS LEVEL FOLLOWERS
# for value in rts[int(row['id_'])]:
                                post_user =row['user_id']
                                retweeters.append(post_user)
                                upper_level = set([post_user])
                                flag = False
                                iter_=1
                                users.append(post_user)
                                print(post_user)
                                for value in foll_id[int(row['user_id'])]:
#                                     print(post_user, value)
                                    try:
        
                                        G.add_edge(post_user , value)
                                    except:
                                        pass
                                
#                                 print("before loop: {}".format(foll_id[int(post_user)]))
                                
                                while(not flag and tobecovered):
                                    print("iter: {}".format(iter_))
                                    iter_+=1
                                    flag=True
                                    nowcovered=set()
                                    for each_user in upper_level:
#                                         print("eu",'3860520309' in upper_level)
                                        if int(each_user) in foll_id:
                                            for each_follower in foll_id[int(each_user)]:
#                                                 print("ef",'3860520309' == each_follower)
                                                if each_follower in tobecovered:
                                                    nowcovered.add(each_follower)
                                                    flag=False
                                                    if each_user in retweeters and each_follower in retweeters :
                                                        G.add_edge(each_user , each_follower)
#                                                         print("g1",'3860520309' in G1)
                                                    else:
                                                        pass
#                                                         print(each_user , each_follower , "Not a retweeter")
                                    for each_covered in nowcovered:
#                                         if each_covered in tobecovered:        
                                        tobecovered.remove(each_covered)
                                    upper_level = copy.deepcopy(nowcovered)
#                                 print(flag , len(tobecovered),"after while")
                                
                                if tobecovered:
#                                     print("tbd",'3860520309' in tobecovered)
                                    upper_level = copy.deepcopy(tobecovered)
                                    flag = False
                                    iterg_=1
    #                                 print("before loop: {}".format(foll_id[int(post_user)]))

                                    while(not flag and tobecovered):
                                        print("iter: {}".format(iterg_))
                                        iterg_+=1
                                        flag=True
                                        nowcovered=set()
                                        for each_user in upper_level:
#                                             print("eu2",'3860520309' in upper_level)
                                            
                                            if int(each_user) in foll_id:
#                                                 print("ef2",'3860520309' in foll_id[int(each_user)])
                                                for each_follower in foll_id[int(each_user)]:
                                                    if each_follower in tobecovered:
#                                                         print("rf3",'3860520309'==each_follower)
                                                        nowcovered.add(each_follower)
                                                        flag=False
                                                        if each_user in retweeters and each_follower in retweeters :
                                                            G.add_edge(each_user , each_follower)
#                                                             print("g2",'3860520309' in G2)
                                                        else:
                                                            pass
#                                                       ``      print(each_user , each_follower , "Not a retweeter G2")
                                        for each_covered in nowcovered:
#                                             if each_covered in tobecovered:
                                            tobecovered.remove(each_covered)
                                        upper_level = copy.deepcopy(nowcovered)
#                                     print(flag , len(tobecovered),"after 2nd while")
                                    
                                    
#                             g1_nodes = list(G1.nodes())
#                             g2_nodes = list(G2.nodes())
#                             for g in  g1_nodes :
#                                 if g not in retweeters:
# #                                     print("g1r",g=='3860520309')
#                                     G1.remove_node(g)
                            
#                             for g1 in g2_nodes:
#                                 if g1 not in retweeters:
# #                                     print("g2r",g1=='3860520309')
#                                     G2.remove_node(g1)
# #                                 print("yes")
                            
# #                             print(set(retweeters) - set(g1_nodes+g2_nodes+list(tobecovered)), "CHECK")
            

#                             roots = (v for v, d in G1.in_degree() if d == 0)
#                             leaves = (v for v, d in G1.out_degree() if d == 0)

#                             for root in roots:
# #                                 print("Root:",'3860520309'==root)
#                                 for leaf in leaves:
# #                                     print("Leaf",leaf)
# #                                     print("leaf:",'3860520309'==leaf)
#                                     if leaf in retweeters:
#                                         paths=[]
#                                         paths = nx.all_simple_paths(G1, root, leaf)
# #                                         print(paths)
#                                         all_paths.extend(paths)
# #                                         print(all_paths)
#     #                                 break
#                             print("Actual cacades:" , len(all_paths))
        
        
#                             roots1 = (v for v, d in G2.in_degree() if d == 0)
#                             leaves1 = (v for v, d in G2.out_degree() if d == 0)

#                             for root in roots1:
# #                                 print("Root:",root)
# #                                 print("Root:",'3860520309'==root)
#                                 for leaf in leaves1:
# #                                     print("leaf:",'3860520309'==leaf)
# #                                     print("Leaf",leaf)
#                                     if leaf in retweeters:
#                                         paths=[]
#                                         paths = nx.all_simple_paths(G2, root, leaf)
# #                                         print(paths)
#                                         all_path2.extend(paths)
# #                                         print(all_path2)
#     #                                 break
# #                             max_+=len(all_path2)
# #                             count_+=1
# #                             print(max_)
#     #                             break
#     #                             dummy_path = all_path2 + list(tobecovered)
#     #                             print("Dummy cascade" , len(dummy_path))
# #                             print(dummy_path)

                            
#                             for actual in all_paths:
#                                 final_test_cascade.append(actual)
#                                 if len(all_path2)==0  :
#                                     final_cascade.append(actual)
#                                 else:
#                                     for dummy in all_path2:
#                                         temp=list()
#                                         temp.extend(actual)
#                                         temp.extend(dummy)
# #                                         print(len(temp),"length")
#                                         m1+= len(temp)
#                                         final_cascade.append(temp)
                                        
# #                             arr = rts_time[int(row['id_'])]
# #                             for path in final_cascade:
# #                                     if len(path)>4:
# #                                         print(path,"path" )
# #                                         temp={}
# #                                         for element in path[1:]:
# # #                                             print(arr.keys())
# #                                             print(len(arr.keys()) ," ret time")
# #                                             try:
# #                                                 temp[element]=arr[element]
# #                                             except:
# #                                                 print("left missing")
# # #                                                 pass
# #                                         sorted_temp ={k: v for k, v in sorted(temp.items(), key=lambda item: item[1])}
# #                                         inner_data=[path[0]]
# #                                         [inner_data.extend([k,v]) for k,v in sorted_temp.items()]
# #                                         final_data.append(inner_data)
                
#                             del(G1)
#                             del(G2)

#                             break
#                         call uthalo

2826529099
iter: 1
iter: 2
iter: 1
1076486409542873088
iter: 1
iter: 2
iter: 1
2392031700
iter: 1
iter: 2
iter: 1
148028414
iter: 1
iter: 1
iter: 2
279483060
iter: 1
iter: 2
iter: 1
2937105535
iter: 1
iter: 2
iter: 3
iter: 1
368306365
iter: 1
iter: 2
iter: 1
1200095151751778309
iter: 1
iter: 2
iter: 3
iter: 1
1021495661013942278
iter: 1
iter: 2
iter: 1
1180114533777494017
iter: 1
iter: 2
iter: 1
822244260175089664
iter: 1
iter: 2
iter: 1
4358041572
iter: 1
iter: 2
iter: 1
1100927498
iter: 1
iter: 2
iter: 1
822392302224052224
iter: 1
iter: 2
iter: 1
iter: 2
2283318716
iter: 1
iter: 2
iter: 1
743275561
iter: 1
iter: 2
iter: 1
iter: 2
1213812904413122563
iter: 1
iter: 2
iter: 3
iter: 1
1109163883082637312
iter: 1
iter: 2
iter: 1
140372853
iter: 1
iter: 2
iter: 1
iter: 2
1042572838761201665
iter: 1
iter: 2
iter: 1
iter: 2
859309618270322688
iter: 1
iter: 2
iter: 1
279483060
iter: 1
iter: 2
iter: 1
1965152238
iter: 1
iter: 1
iter: 2
175372810
iter: 1
iter: 2
iter: 1
2540896328
iter: 1
iter:

In [61]:
len(users)

76

In [60]:
print(len(G.nodes) , len(G.edges()))

2973809 3186508


In [62]:
nx.write_gpickle(G , "final_complete_graph_jamia_final2")

# CASCADE CREATION

In [43]:
# G= nx.DiGraph()
import copy
FOLDERNAME1="twitter_Retweeters_history"
FOLDERNAME2="twitter_Followers_history"
rts_dataset=dict()
total=0
labels=[]
final_data=[]

final_cascade =[]
final_test_cascade=[]

m1=0
for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
    for fname in files:
#         print(fname)
        if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
            filepath=root+"/"+fname
            search_file=filepath.split("/")
#             print("here")
            if len(search_file)>7:
#                 print(search_file[7])
#                 tweets=dict()
                with open(filepath) as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
#                         print("====Row===={}".format(row))
                        inner={}
                        if  int(row['retweet_count'])>5:
#                             row['id_'] in tweet_features and
                            G1= nx.DiGraph()
                            G2 = nx.DiGraph()
                            all_paths = []
                            all_path2=[]
                            retweeters=[]
                            if int(row['user_id']) in foll_id  and int(row['id_']) in rts_time:
                                retweeters=[]
                                retweeters = rts[int(row['id_'])]
#                                 print("i",row['user_id'],'3860520309' in retweeters)
#                                 retweeters.append()
                                tobecovered=set()
#                                 print(type(retweeters))
                                tobecovered=set(copy.deepcopy(retweeters))
#                                 print(len(retweeters), "tobecovered vefore")
#                                 print(retweeters)
                                nowcovered=set()
# first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
####FOR FIRS LEVEL FOLLOWERS
# for value in rts[int(row['id_'])]:
                                post_user =row['user_id']
                                retweeters.append(post_user)
                                upper_level = set([post_user])
                                flag = False
                                iter_=1
#                                 print("before loop: {}".format(foll_id[int(post_user)]))
                                
                                while(not flag and tobecovered):
                                    print("iter: {}".format(iter_))
                                    iter_+=1
                                    flag=True
                                    nowcovered=set()
                                    for each_user in upper_level:
#                                         print("eu",'3860520309' in upper_level)
                                        if int(each_user) in foll_id:
                                            for each_follower in foll_id[int(each_user)]:
#                                                 print("ef",'3860520309' == each_follower)
                                                if each_follower in tobecovered:
                                                    nowcovered.add(each_follower)
                                                    flag=False
                                                    if each_user in retweeters and each_follower in retweeters  and each_user !=each_follower:
                                                        G1.add_edge(each_user , each_follower)
#                                                         print("g1",'3860520309' in G1)
                                                    else:
                                                        pass
#                                                         print(each_user , each_follower , "Not a retweeter")
                                    for each_covered in nowcovered:
#                                         if each_covered in tobecovered:        
                                        tobecovered.remove(each_covered)
                                    upper_level = copy.deepcopy(nowcovered)
#                                 print(flag , len(tobecovered),"after while")
                                
                                if tobecovered:
#                                     print("tbd",'3860520309' in tobecovered)
                                    upper_level = copy.deepcopy(tobecovered)
                                    flag = False
                                    iterg_=1
    #                                 print("before loop: {}".format(foll_id[int(post_user)]))

                                    while(not flag and tobecovered):
                                        print("iter: {}".format(iterg_))
                                        iterg_+=1
                                        flag=True
                                        nowcovered=set()
                                        for each_user in upper_level:
#                                             print("eu2",'3860520309' in upper_level)
                                            
                                            if int(each_user) in foll_id:
#                                                 print("ef2",'3860520309' in foll_id[int(each_user)])
                                                for each_follower in foll_id[int(each_user)]:
                                                    if each_follower in tobecovered:
#                                                         print("rf3",'3860520309'==each_follower)
                                                        nowcovered.add(each_follower)
                                                        flag=False
                                                        if each_user in retweeters and each_follower in retweeters and each_user !=each_follower :
                                                            G2.add_edge(each_user , each_follower)
#                                                             print("g2",'3860520309' in G2)
                                                        else:
                                                            pass
#                                                       ``      print(each_user , each_follower , "Not a retweeter G2")
                                        for each_covered in nowcovered:
#                                             if each_covered in tobecovered:
                                            tobecovered.remove(each_covered)
                                        upper_level = copy.deepcopy(nowcovered)
#                                     print(flag , len(tobecovered),"after 2nd while")
                                    
                                    
                                g1_nodes = list(G1.nodes())
                                g2_nodes = list(G2.nodes())
                                for g in  g1_nodes :
                                    if g not in retweeters:
    #                                     print("g1r",g=='3860520309')
                                        G1.remove_node(g)

                                for g1 in g2_nodes:
                                    if g1 not in retweeters:
    #                                     print("g2r",g1=='3860520309')
                                        G2.remove_node(g1)
    #                                 print("yes")

    #                             print(set(retweeters) - set(g1_nodes+g2_nodes+list(tobecovered)), "CHECK")


                                roots = (v for v, d in G1.in_degree() if d == 0)
                                leaves = (v for v, d in G1.out_degree() if d == 0)

                                for root in roots:
    #                                 print("Root:",'3860520309'==root)
                                    for leaf in leaves:
    #                                     print("Leaf",leaf)
    #                                     print("leaf:",'3860520309'==leaf)
                                        if leaf in retweeters:
                                            paths=[]
                                            paths = nx.all_simple_paths(G1, root, leaf)
    #                                         print(paths)
                                            all_paths.extend(paths)
    #                                         print(all_paths)
        #                                 break
                                print("Actual cacades:" , len(all_paths))


                                roots1 = (v for v, d in G2.in_degree() if d == 0)
                                leaves1 = (v for v, d in G2.out_degree() if d == 0)

                                for root in roots1:
    #                                 print("Root:",root)
    #                                 print("Root:",'3860520309'==root)
                                    for leaf in leaves1:
    #                                     print("leaf:",'3860520309'==leaf)
    #                                     print("Leaf",leaf)
                                        if leaf in retweeters:
                                            paths=[]
                                            paths = nx.all_simple_paths(G2, root, leaf)
    #                                         print(paths)
                                            all_path2.extend(paths)
    #                                         print(all_path2)
        #                                 break
    #                             max_+=len(all_path2)
    #                             count_+=1
    #                             print(max_)
        #                             break
        #                             dummy_path = all_path2 + list(tobecovered)
        #                             print("Dummy cascade" , len(dummy_path))
    #                             print(dummy_path)


                                for actual in all_paths:
                                    final_test_cascade.append(actual)
                                    if len(all_path2)==0  :
                                        final_cascade.append(actual)
                                    else:
                                        for dummy in all_path2:
                                            temp=list()
                                            temp.extend(actual)
                                            temp.extend(dummy)
    #                                         print(len(temp),"length")
                                            m1+= len(temp)
                                            final_cascade.append(temp)

                                arr = rts_time[int(row['id_'])]
                            
                                for path in final_cascade:
                                        if len((path and retweeters))>3:
#                                             print(path and retweeters)
#                                             print(set(path)-set(arr.keys()))
#                                             print(path,"path" )
                                            temp={}
                                            for element in (set(path) and set(arr.keys())):
    #                                             print(arr.keys())
#                                                 print(len(arr.keys()) ," ret time")
#                                                 try:
                                                temp[element]=arr[element]
#                                                 except:
#                                                     print("left missing")
    #                                                 pass
#                                             if len(temp.keys())>3:
#                                             print(len(temp.keys()),"temp")
                                            if len(temp.keys())>4:
                                                sorted_temp ={k: v for k, v in sorted(temp.items(), key=lambda item: item[1])}
                                                inner_data=[path[0]]
                                                [inner_data.extend([k,v]) for k,v in sorted_temp.items()]
    #                                                 print(len(final_data))
                                                final_data.append(inner_data)

                            del(G1)
                            del(G2)

#                             break
#                         call uthalo

iter: 1
iter: 2
iter: 1
Actual cacades: 8
iter: 1
iter: 2
iter: 1
Actual cacades: 25
iter: 1
iter: 2
iter: 1
Actual cacades: 10
iter: 1
iter: 1
iter: 2
Actual cacades: 0
iter: 1
iter: 2
iter: 1
Actual cacades: 90
iter: 1
iter: 2
iter: 3
iter: 1
Actual cacades: 0
iter: 1
iter: 2
iter: 1
Actual cacades: 3
iter: 1
iter: 2
iter: 3
iter: 1
Actual cacades: 0
iter: 1
iter: 2
iter: 1
Actual cacades: 47
iter: 1
iter: 2
iter: 1
Actual cacades: 1
iter: 1
iter: 2
iter: 1
Actual cacades: 18
iter: 1
iter: 2
iter: 1
Actual cacades: 1
iter: 1
iter: 2
iter: 1
Actual cacades: 21
iter: 1
iter: 2
iter: 1
iter: 2
Actual cacades: 4
iter: 1
iter: 2
iter: 1
Actual cacades: 8
iter: 1
iter: 2
iter: 1
iter: 2
Actual cacades: 6
iter: 1
iter: 2
iter: 3
iter: 1
Actual cacades: 55
iter: 1
iter: 2
iter: 1
Actual cacades: 18
iter: 1
iter: 2
iter: 1
iter: 2
Actual cacades: 3
iter: 1
iter: 2
iter: 1
iter: 2
Actual cacades: 2
iter: 1
iter: 2
iter: 1
Actual cacades: 5
iter: 1
iter: 2
iter: 1
Actual cacades: 68
iter: 1
ite

In [8]:
print(len(final_cascade), len(final_test_cascade))
# final_cascade
print(m1/len(final_cascade))
# print(int(3860520309) in foll_id)
# print(int(3860520309) in rts_time)

1055 1055
0.0


In [52]:
final_data[0]

['2826529099',
 '1138786355218145281',
 '2020-02-20 07:11:14',
 '448764973',
 '2020-02-20 07:11:50',
 '3903318556',
 '2020-02-20 07:12:00',
 '954805504617758720',
 '2020-02-20 07:15:33',
 '2270213274',
 '2020-02-20 07:15:38',
 '787467474',
 '2020-02-20 07:17:25',
 '863987277046857728',
 '2020-02-20 07:17:43',
 '2328865356',
 '2020-02-20 07:29:09',
 '785904386799120384',
 '2020-02-20 08:04:59',
 '42779085',
 '2020-02-20 09:09:27']

In [48]:
import pickle
with open('final_data_deep_diffuse_sorted_jamia.pkl', 'wb') as f:
    pickle.dump(final_data, f)

In [49]:
len(final_data)

45556

In [121]:
m1/len(final_cascade)

4.047276843944577

In [63]:
import pickle
with open('user_sir.pkl', 'wb') as f:
    pickle.dump(users, f)
    
# with open('test_cascade_covid_final.pkl', 'wb') as f:
#     pickle.dump(final_test_cascade, f)

In [42]:
# for each in final_cascade[1587045]:
#     print(each)
# # print(len(final_cascade[5000]))
# # print(len(set(final_cascade[5000])))

1213812904413122563
1146802871431655428
935142655213703168
400184218


In [66]:
len(final_test_cascade)

8

In [25]:
# G= nx.DiGraph()
import copy
FOLDERNAME1="twitter_Retweeters_history"
FOLDERNAME2="twitter_Followers_history"
rts_dataset=dict()
total=0
labels=[]
final_data=[]

final_cascade =[]
final_test_cascade=[]

m1=0
for (root,dirs,files) in os.walk('/home/sakshi18013/Documents/new_data/Thesis/data/#jamiaviolence'):
    for fname in files:
#         print(fname)
        if fname.endswith(".csv") and fname =='twitterDatabase_hate_screen.csv':
            filepath=root+"/"+fname
            search_file=filepath.split("/")
#             print("here")
            if len(search_file)>7:
#                 print(search_file[7])
#                 tweets=dict()
                with open(filepath) as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
#                         print("====Row===={}".format(row))
                        inner={}
                        if  int(row['retweet_count'])>5:
#                             row['id_'] in tweet_features and
                            G1= nx.DiGraph()
                            G2 = nx.DiGraph()
                            all_paths = []
                            all_path2=[]
                            retweeters=[]
                            if int(row['user_id']) in foll_id  and int(row['id_']) in rts_time:
                                retweeters=[]
                                retweeters = rts[int(row['id_'])]
#                                 print("i",row['user_id'],'3860520309' in retweeters)
#                                 retweeters.append()
                                tobecovered=set()
#                                 print(type(retweeters))
                                tobecovered=set(copy.deepcopy(retweeters))
#                                 print(len(retweeters), "tobecovered vefore")
#                                 print(retweeters)
                                nowcovered=set()
                                print(retweeters)
#                                 break

            
                                cascades=[]
                                for i in range(len(retweeters)+1):
                                    for j in range(i+1 , len(retweeters)+1):
                                        sub = retweeters[i:j]
                                        if len(sub)>4:
#                                             print(appending)
                                            cascades.append(sub)
                                final_data.append(cascades)
#                                 print(len(final_data))
#                                 break
                                        
                                
# first_retweets=[]
#                                 print("level1 passed",row['id_'],int(row['retweet_count']) , " followers:" , len(foll_id[int(row['user_id'])]))                       
            
####FOR FIRS LEVEL FOLLOWERS
# for value in rts[int(row['id_'])]:
                                

#                             break
#                         call uthalo

['42779085', '785904386799120384', '2328865356', '863987277046857728', '787467474', '2270213274', '954805504617758720', '3903318556', '448764973', '1138786355218145281', '2826529099', '2826529099', '2826529099', '2826529099', '2826529099', '2826529099', '2826529099']
['1171452867313463298', '16804541', '361913249', '75605073', '724590194138046465', '1118713226160553985', '614908818', '2307712298', '864952746285252608', '2698033460', '795681622364450816', '1146249629509439488', '948737821262544901', '1215642777611976709', '888069606161735681', '2178214820', '1164003523442302976', '926698045701464065', '1857092954', '1185580744095092737', '731439265092128768', '837516459987386369', '923892620178235392', '1190683107160051712', '44616260', '1165358470129954816', '3232492459', '947173998709817345', '1677723938', '2307132318', '1071999249368309760', '74810575', '1076486409542873088', '1076486409542873088', '1076486409542873088', '1076486409542873088']
['873369085496741889', '416397821', '122

['61524466', '88180867', '2510709925', '73610851', '2480164896', '593779745', '61524466', '88180867', '2510709925', '73610851', '2480164896', '593779745', '96108332', '96108332', '96108332', '96108332']
['4852468883', '1057191361642807298', '421670304', '1119085664849367040', '1107986001069309952', '1205549950374834176', '1212252598423523336', '1206235241628753920', '1206235241628753920', '1206235241628753920', '1206235241628753920']
['280059417', '766136448831586304', '1096566492', '107382271', '4030371792', '1974029372', '939421760851623937', '72819765', '86620692', '2385051428', '817232983417647104', '1209533914043142144', '100445804', '841047553', '114015079', '1342399172', '1222547213739388928', '1229061760109273088', '52999499', '41798085', '550854560', '852402260613238785', '2534141970', '44846776', '790972165415841792', '54476164', '1089848916009517056', '1123195000550858753', '1214940116', '2772554473', '1109458475371909120', '3579200054', '3579200054', '3579200054', '35792000

In [27]:
len(final_data)

76

In [28]:
import pickle
with open('final_data_deep_diffuse_jamia.pkl', 'wb') as f:
    pickle.dump(final_data, f)

In [44]:
# roots = (v for v, d in G.in_degree() if d == 0)
# leaves = (v for v, d in G.out_degree() if d == 0)
# all_paths = []
# for root in roots:
#     print(root)
#     for leaf in leaves:
#         paths = nx.all_simple_paths(G, root, leaf)
#         all_paths.extend(paths)
#         break
# all_paths